In [74]:
import pandas as pd
import numpy as np

In [75]:
data=pd.read_csv('root2ai - Data.csv')

In [76]:
data['Target'].unique()

array(['Blockchain', 'Bigdata', 'credit reporting', 'Cyber Security',
       'FinTech', 'Microservices', 'Neobanks', 'Robo Advising',
       'Stock Trading', 'Data Security', 'Reg Tech'], dtype=object)

In [77]:
data

,Text,Target
0,reserve bank forming expert committee based in...,Blockchain
1,director could play role financial system,Blockchain
2,preliminary discuss secure transaction study r...,Blockchain
3,security indeed prove essential transforming f...,Blockchain
4,bank settlement normally take three days based...,Blockchain
...,...,...
22699,fourth study discusses blockchain technology e...,Reg Tech
22700,book finishes stating biggest issue emerging F...,Reg Tech
22701,people culture cess,Reg Tech
22702,author challenges execu tive lead change stop ...,Reg Tech


In [78]:
import re

In [79]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tejashwini\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [80]:
corpus=[]

In [81]:
for i in range(0,len(data)):
    review = re.sub('[^a-zA-Z]',' ',str(data['Text'][i]))
    review = review.lower()
    review = review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [82]:
data

,Text,Target
0,reserve bank forming expert committee based in...,Blockchain
1,director could play role financial system,Blockchain
2,preliminary discuss secure transaction study r...,Blockchain
3,security indeed prove essential transforming f...,Blockchain
4,bank settlement normally take three days based...,Blockchain
...,...,...
22699,fourth study discusses blockchain technology e...,Reg Tech
22700,book finishes stating biggest issue emerging F...,Reg Tech
22701,people culture cess,Reg Tech
22702,author challenges execu tive lead change stop ...,Reg Tech


In [83]:
from sklearn.feature_extraction.text import CountVectorizer 

In [84]:
cv=CountVectorizer()
X=cv.fit_transform(corpus).toarray()

In [85]:
y=data.iloc[:,-1:]

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [87]:
import tensorflow as tf

In [88]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM

In [89]:
mnist=tf.keras.datasets.mnist
(X_train,y_train),(X_test,y_test)=mnist.load_data()

In [90]:
print(X_train.shape)
print(X_train[0].shape)

(60000, 28, 28)
(28, 28)


In [91]:
X_train=X_train/255.0
X_test=X_test/255.0

In [92]:
model=Sequential()

In [93]:
model.add(LSTM(128,input_shape=(X_train.shape[1:]),activation='relu',return_sequences=True))
model.add(Dropout(0.3))

In [94]:
model.add(LSTM(128,activation='relu'))
model.add(Dropout(0.3))

In [95]:
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.3))

In [96]:
model.add(Dense(10,activation='softmax'))

In [97]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 28, 128)           80384     
_________________________________________________________________
dropout_6 (Dropout)          (None, 28, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [98]:
opt=tf.keras.optimizers.Adam(lr=1e-3,decay=1e-5)

In [99]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

In [100]:
history=model.fit(X_train,y_train,epochs=3,validation_split=0.2)

Epoch 1/3
1500/1500 [==============================] - 49s 33ms/step - loss: 0.8910 - accuracy: 0.7060 - val_loss: 0.5830 - val_accuracy: 0.8827
Epoch 2/3
1500/1500 [==============================] - 55s 37ms/step - loss: 0.2420 - accuracy: 0.9374 - val_loss: 0.1439 - val_accuracy: 0.9606
Epoch 3/3
1500/1500 [==============================] - 58s 39ms/step - loss: 0.1661 - accuracy: 0.9580 - val_loss: 0.0959 - val_accuracy: 0.9751


Epoch 1/3
1500/1500 [==============================] - 53s 35ms/step - loss: 0.8229 - accuracy: 0.7261 - val_loss: 0.2814 - val_accuracy: 0.9096
Epoch 2/3
1500/1500 [==============================] - 67s 45ms/step - loss: 0.2850 - accuracy: 0.9216 - val_loss: 0.1180 - val_accuracy: 0.9662
Epoch 3/3
1500/1500 [==============================] - 68s 45ms/step - loss: 0.1661 - accuracy: 0.9575 - val_loss: 0.1023 - val_accuracy: 0.9718


In [101]:
y_pred=model.predict_classes(X_test)

In [102]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9757

In [105]:
from sklearn.metrics import precision_score
precision_score(y_test,y_pred,average='micro')

0.9757